In [1]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [458 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [980 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,241 kB]
Hit:10 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [860 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,136 kB]
Hit:13 h

In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:


from pyspark.sql.functions import round, col



In [4]:


# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get('home_sales_revised.csv'), sep=',', header=True)

# display the DataFrame
df.show()



+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 2. Create a temporary view of the DataFrame.
df.createOrReplaceTempView('home_sales')

df.printSchema()



root
 |-- id: string (nullable = true)
 |-- date: string (nullable = true)
 |-- date_built: string (nullable = true)
 |-- price: string (nullable = true)
 |-- bedrooms: string (nullable = true)
 |-- bathrooms: string (nullable = true)
 |-- sqft_living: string (nullable = true)
 |-- sqft_lot: string (nullable = true)
 |-- floors: string (nullable = true)
 |-- waterfront: string (nullable = true)
 |-- view: string (nullable = true)



In [6]:


# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

spark.sql("""SELECT date_built,
    ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE bedrooms = 4
    GROUP BY date_built
""").show()



+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2016|    296050.24|
|      2012|    298233.42|
|      2017|    296576.69|
|      2014|    299073.89|
|      2013|    299999.39|
|      2011|     302141.9|
|      2015|    307908.86|
|      2010|    296800.75|
+----------+-------------+



In [7]:
# filter the houses with 4 bedrooms
four_b_house = df.filter(df["bedrooms"] == 4)

# group by the year built and getting the average
four_b_price_by_year = four_b_house.groupBy("date_built").agg({"price" : "avg"})

# round the avg(price) by two decimals
four_b_price = four_b_price_by_year.withColumn('rounded avg(price)',
                                               round((four_b_price_by_year['avg(price)']), 2))

# show only the date_built and the rounded average price
four_b_price.select(['date_built', 'rounded avg(price)']).show()

+----------+------------------+
|date_built|rounded avg(price)|
+----------+------------------+
|      2016|         296050.24|
|      2012|         298233.42|
|      2017|         296576.69|
|      2014|         299073.89|
|      2013|         299999.39|
|      2011|          302141.9|
|      2015|         307908.86|
|      2010|         296800.75|
+----------+------------------+



In [8]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3
# bathrooms rounded to two decimal places?

spark.sql("""SELECT date_built,
    ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE bedrooms = 3 AND  bathrooms = 3
    GROUP BY date_built
""").show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2016|    290555.07|
|      2012|    293683.19|
|      2017|    292676.79|
|      2014|    290852.27|
|      2013|    295962.27|
|      2011|    291117.47|
|      2015|     288770.3|
|      2010|    292859.62|
+----------+-------------+



In [9]:
# filter the houses with 3 bedrooms 3 bathrooms
three_house = df.filter(df["bedrooms"] == 3)
three_bb_house = three_house.filter(three_house['bathrooms'] ==3)

# group by the year built and getting the average
three_bb_price_by_year = three_bb_house.groupBy("date_built").agg({"price" : "avg"})

# round the avg(price) by two decimals
three_bb_price = three_bb_price_by_year.withColumn('rounded avg(price)',
                                                   round((three_bb_price_by_year['avg(price)']), 2))

# show only the date_built and the rounded average price
three_bb_price.select(['date_built', 'rounded avg(price)']).show()

+----------+------------------+
|date_built|rounded avg(price)|
+----------+------------------+
|      2016|         290555.07|
|      2012|         293683.19|
|      2017|         292676.79|
|      2014|         290852.27|
|      2013|         295962.27|
|      2011|         291117.47|
|      2015|          288770.3|
|      2010|         292859.62|
+----------+------------------+



In [10]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

spark.sql("""SELECT date_built,
    ROUND(AVG(price), 2) AS average_price
    FROM home_sales
    WHERE bedrooms = 3 AND  bathrooms = 3 AND floors = 2 AND sqft_living >= 2000
    GROUP BY date_built
""").show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2016|     293965.1|
|      2012|    307539.97|
|      2017|    280317.58|
|      2014|    298264.72|
|      2013|    303676.79|
|      2011|    276553.81|
|      2015|    297609.97|
|      2010|    285010.22|
+----------+-------------+



In [11]:
# filter the houses with 3 bedrooms 3 bathrooms, 2 floors, and >= 2000sqf
three_b_house = df.filter((df["bedrooms"] == 3) & (df['bathrooms'] ==3) & (df['floors'] == 2))
three_bbfsqft_house = three_b_house.filter(three_b_house['sqft_living'] >= 2000)

# group by the year built and getting the average
three_bbfsqft_price_by_year = three_bbfsqft_house.groupBy("date_built").agg({"price" : "avg"})

# round the avg(price) by two decimals
three_bbfsqft_price = three_bbfsqft_price_by_year.withColumn('rounded avg(price)',
                                                             round((three_bbfsqft_price_by_year['avg(price)']), 2))

# show only the date_built and the rounded average price
three_bbfsqft_price.select(['date_built', 'rounded avg(price)']).show()

+----------+------------------+
|date_built|rounded avg(price)|
+----------+------------------+
|      2016|          293965.1|
|      2012|         307539.97|
|      2017|         280317.58|
|      2014|         298264.72|
|      2013|         303676.79|
|      2011|         276553.81|
|      2015|         297609.97|
|      2010|         285010.22|
+----------+------------------+



In [12]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

spark.sql("""SELECT ROUND(AVG(view), 2) AS average_view
    FROM home_sales
    WHERE price >= 350000
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+------------+
|average_view|
+------------+
|       32.26|
+------------+

--- 0.4127004146575928 seconds ---


In [13]:
start_time = time.time()

# houses greater or equal to $350,000
vh_price = df.filter(df["price"] >= 350000)

# getting the average of view
vh_price_avg = vh_price.agg({"view" : "avg"})

# round the avg(view) by two decimals
vh = vh_price_avg.withColumn('rounded avg(view)', round((vh_price_avg['avg(view)']), 2))

# show only the data with rounded decimals
vh.select(['rounded avg(view)']).show()

print("--- %s seconds ---" % (time.time() - start_time))

+-----------------+
|rounded avg(view)|
+-----------------+
|            32.26|
+-----------------+

--- 0.30066847801208496 seconds ---


In [18]:


# 7. Cache the temporary table home_sales

spark.catalog.cacheTable("home_sales")




In [20]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [22]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.
start_time = time.time()

spark.sql("""SELECT ROUND(AVG(view), 2) AS average_view
    FROM home_sales
    WHERE price >= 350000
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+------------+
|average_view|
+------------+
|       32.26|
+------------+

--- 0.1875460147857666 seconds ---


In [23]:
start_time = time.time()

# houses greater or equal to $350,000
vh_price = df.filter(df["price"] >= 350000)

# getting the average of view
vh_price_avg = vh_price.agg({"view" : "avg"})

# round the avg(view) by two decimals
vh = vh_price_avg.withColumn('rounded avg(view)', round((vh_price_avg['avg(view)']), 2))

# show only the data with rounded decimals
vh.select(['rounded avg(view)']).show()

print("--- %s seconds ---" % (time.time() - start_time))

+-----------------+
|rounded avg(view)|
+-----------------+
|            32.26|
+-----------------+

--- 0.23142194747924805 seconds ---


In [24]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data

df.write.partitionBy("date_built").parquet("hs_partitioned", mode ="overwrite")

In [25]:
# 11. Read the parquet formatted data.
hsp_df = spark.read.parquet('hs_partitioned')

In [26]:
# 12. Create a temporary table for the parquet data.
spark.sql('uncache table home_sales')

DataFrame[]

In [27]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

spark.sql("""SELECT ROUND(AVG(view), 2) AS average_view
    FROM home_sales
    WHERE price >= 350000
""").show()

print("--- %s seconds ---" % (time.time() - start_time))

+------------+
|average_view|
+------------+
|       32.26|
+------------+

--- 0.2617645263671875 seconds ---


In [28]:
# 14. Uncache the home_sales temporary table.
spark.sql('uncache table home_sales')

DataFrame[]

In [29]:
# 15. Check if the home_sales is no longer cached
spark.catalog.isCached('home_sales')

False